In [87]:
# Import dependencies
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

# Create global seed
yogi = 8

In [88]:
# Read in data
file_path = Path('Data/Saved/pitching_all.csv')
# file_path = Path('Data/Statcast/Statcast_2020.csv')
df_pitch = pd.read_csv(file_path, index_col = 'Unnamed: 0')
df_pitch.head()

C:\Users\Matthew\anaconda3\envs\mlenv\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,type,pitch_type,release_speed,effective_speed,release_spin_rate,release_pos_x,release_pos_z,zone,stand,p_throws,pitch_number,if_fielding_alignment,of_fielding_alignment,spin_axis
0,X,FF,98.8,100.2,2483.0,-0.41,6.46,14.0,L,R,2,Strategic,Standard,198.0
1,B,FF,98.7,100.0,2522.0,-0.33,6.62,12.0,L,R,1,Strategic,Standard,189.0
2,S,SL,89.6,90.8,2537.0,-0.10,6.79,5.0,R,R,6,Infield shift,Standard,58.0
3,S,FF,100.4,101.7,2469.0,-0.38,6.50,6.0,R,R,5,Infield shift,Standard,191.0
4,S,FF,97.6,98.9,2339.0,-0.18,6.63,12.0,R,R,4,Infield shift,Standard,191.0


### Preprocessing Data

In [89]:
# Generate categorical variable list
pitch_cat = df_pitch.dtypes[df_pitch.dtypes == 'object'].index.tolist()
pitch_cat.remove('type')
pitch_cat

['pitch_type',
 'stand',
 'p_throws',
 'if_fielding_alignment',
 'of_fielding_alignment']

In [90]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_pitch[pitch_cat]))

# Add the encoded varibale names to the DataFrame
encode_df.columns = enc.get_feature_names(pitch_cat)
encode_df.head()

,pitch_type_CH,pitch_type_CS,pitch_type_CU,pitch_type_EP,pitch_type_FA,pitch_type_FC,pitch_type_FF,pitch_type_FO,pitch_type_FS,pitch_type_FT,...,stand_R,p_throws_L,p_throws_R,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_4th outfielder,of_fielding_alignment_Extreme outfield shift,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [91]:
# Merge one-hot encoded features and drop the originals
df_pitch = df_pitch.merge(
    encode_df,
    left_index = True,
    right_index = True
).drop(pitch_cat, 1)

df_pitch.head()

,type,release_speed,effective_speed,release_spin_rate,release_pos_x,release_pos_z,zone,pitch_number,spin_axis,pitch_type_CH,...,stand_R,p_throws_L,p_throws_R,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_4th outfielder,of_fielding_alignment_Extreme outfield shift,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,X,98.8,100.2,2483.0,-0.41,6.46,14.0,2,198.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,S,91.1,89.9,2347.0,-2.16,6.08,14.0,5,187.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,S,92.2,94.1,2629.0,-1.97,6.26,12.0,4,164.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,X,91.9,91.0,2422.0,-1.19,5.99,1.0,6,183.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,B,98.7,100.0,2522.0,-0.33,6.62,12.0,1,189.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [92]:
# Drop superfluous columns 'stand_L' and 'p_throws_L'
df_pitch.drop(columns = ['stand_L', 'p_throws_L'], axis = 1, inplace = True)
df_pitch.head()

,type,release_speed,effective_speed,release_spin_rate,release_pos_x,release_pos_z,zone,pitch_number,spin_axis,pitch_type_CH,...,pitch_type_SL,stand_R,p_throws_R,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_4th outfielder,of_fielding_alignment_Extreme outfield shift,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,X,98.8,100.2,2483.0,-0.41,6.46,14.0,2,198.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,S,91.1,89.9,2347.0,-2.16,6.08,14.0,5,187.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,S,92.2,94.1,2629.0,-1.97,6.26,12.0,4,164.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,X,91.9,91.0,2422.0,-1.19,5.99,1.0,6,183.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,B,98.7,100.0,2522.0,-0.33,6.62,12.0,1,189.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


### Compiling, Training, and Testing Data

In [93]:
# Define features set
X = df_pitch.copy()
X.drop(columns = ['type'], axis = 1, inplace = True)
X.head()

,release_speed,effective_speed,release_spin_rate,release_pos_x,release_pos_z,zone,pitch_number,spin_axis,pitch_type_CH,pitch_type_CS,...,pitch_type_SL,stand_R,p_throws_R,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_4th outfielder,of_fielding_alignment_Extreme outfield shift,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,98.8,100.2,2483.0,-0.41,6.46,14.0,2,198.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,91.1,89.9,2347.0,-2.16,6.08,14.0,5,187.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,92.2,94.1,2629.0,-1.97,6.26,12.0,4,164.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
0,91.9,91.0,2422.0,-1.19,5.99,1.0,6,183.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,98.7,100.0,2522.0,-0.33,6.62,12.0,1,189.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [94]:
# Define target vector
y = df_pitch['type'].values
y[:5]

array(['X', 'S', 'S', 'X', 'B'], dtype=object)

In [95]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = yogi, stratify = y)

In [96]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Find best learning rate

In [97]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
best_rate = 0

for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(
        n_estimators = 20,
        learning_rate = learning_rate,
        max_features = 7,
        max_depth = 4,
        random_state = yogi)
    
    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print(f'Learning Rate: {learning_rate}')

    # Score the model
    print('Accuracy Score (training): {0:.3f}'.format(
        classifier.score(
            X_train_scaled,
            y_train
        )
    ))
    print('Accuracy Score (validation): {0:.3f}'.format(
        classifier.score(
            X_test_scaled,
            y_test
        )
    ))
    print()

    if classifier.score(X_test_scaled, y_test) > best_rate:
        best_rate = learning_rate



Learning Rate: 0.05
Accuracy Score (training): 0.671
Accuracy Score (validation): 0.672



### Create model with best learning rate


In [ ]:
# Choose a learning rate and create classifiers
classifier = GradientBoostingClassifier(
    n_estimators = 20,
    learning_rate = best_rate,
    max_features = 5,
    max_depth = 3,
    random_state = yogi
)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({'Prediction': predictions, 'Actual': y_test}).head(20)

,Prediction,Actual
0,S,S
1,B,B
2,B,B
3,B,B
4,S,S
5,B,B
6,B,B
7,B,B
8,S,S
9,S,S


### Evaluate Model

In [ ]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.6798523691384116


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm
)

# Display confusion matrix
display(cm_df)

,0,1,2
0,22872,1305,7
1,8685,21099,432
2,2212,8264,422


In [ ]:
# Generate classification report
print('Classification Report')
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           B       0.68      0.95      0.79     24184
           S       0.69      0.70      0.69     30216
           X       0.49      0.04      0.07     10898

    accuracy                           0.68     65298
   macro avg       0.62      0.56      0.52     65298
weighted avg       0.65      0.68      0.63     65298

